<a href="https://colab.research.google.com/github/sakshith1/Chat-bot/blob/main/stock2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv('/content/BTC_1Y_graph_coinmarketcap.csv')

# Select the 'close' column (replace 'close' with the actual column name)
data = df['close'].values.reshape(-1, 1)

# Scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define the sequence length for input
sequence_length = 10

# Create sequences of data
sequences = []
targets = []
for i in range(sequence_length, len(scaled_data)):
    sequences.append(scaled_data[i-sequence_length:i, 0])
    targets.append(scaled_data[i, 0])

# Convert lists to NumPy arrays
X = np.array(sequences)
y = np.array(targets)

# Reshape input to 3D (samples, time steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and test sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Create an LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Rescale the predicted and actual values
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


KeyError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('/content/TSLA.csv')

# Preprocess the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'volume']].values)

# Define the time frame for labeling (e.g., 1 day)
time_frame = 1

# Calculate the percentage change in stock price
price_change = df['close'].pct_change(time_frame)
price_change = price_change.fillna(0)  # Replace NaN values with 0

# Define the threshold for labeling
threshold = 0.01  # 1% threshold for price increase

# Label the instances
labels = np.where(price_change > threshold, 1, 0)

# Split the data into training and test sets
split_index = int(0.8 * len(scaled_data))
X_train = scaled_data[:split_index]
X_test = scaled_data[split_index:]
y_train = labels[time_frame:split_index]
y_test = labels[split_index + time_frame:]

# Reshape the input data to match the input shape of the LSTM model
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1, X_train.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Use the trained model to predict the labels for the test data
y_pred = model.predict_classes(X_test)

# Evaluate the model
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy}")

# Plot the actual labels and the predicted labels
plt.figure(figsize=(12, 6))
timestamp = df['timestamp'].iloc[split_index + time_frame:split_index + time_frame + len(y_test)]
plt.plot(timestamp, y_test, label='Actual')
plt.plot(timestamp, y_pred, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Label')
plt.title('Actual vs. Predicted Labels')
plt.legend()
plt.show()


KeyError: ignored